In [2]:
import os, time, sys
from pathlib import Path

SESSION_NAME = "LegoTest"

ROOT = Path().absolute().parent
RAW_INPUT_PATH = Path(f"{ROOT}/data/1_raw_input")
INPUT_PATH = Path(f"{ROOT}/data/2_input")
RESULTS_PATH = Path(f"{ROOT}/data/3_results")

SESSION_ID = SESSION_NAME if SESSION_NAME else str(int(time.time()))
SESSION_PATH = Path(f"{RESULTS_PATH}/{SESSION_ID}")

!mkdir -p {RAW_INPUT_PATH}
!mkdir -p {INPUT_PATH}
!mkdir -p {SESSION_PATH}

print(f"Created new session with ID {SESSION_ID} under {RESULTS_PATH}")

Created new session with ID LegoTest under /mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/data/3_results


In [8]:
##### Data Preparation ( raw input -> input folder)
%cd {ROOT}

raw_input_files = [x for x in RAW_INPUT_PATH.glob('**/*') if x.is_file()]
print(f"found {len(raw_input_files)} raw input files")
_f = "/mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/data/1_raw_input/lego.mp4" #str(raw_input_files[0])
print(_f)
DOWNSAMPLING_RATE = 2

#!bash scripts/third_party/neuralangelo/run_ffmpeg.sh {{SESSION_NAME}} {{_f}} 2
! ffmpeg -i {_f} -vf "select=not(mod(n\,{DOWNSAMPLING_RATE}))" -vsync vfr -q:v 2 {INPUT_PATH}/%06d.jpg

/mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline
found 1 raw input files
/mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/data/1_raw_input/lego.mp4
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable

In [2]:
# SPARSE RECONSTRUCTION: Step 1/3
import pycolmap

COLMAP_SFM_PATH = Path(f"{SESSION_PATH}/sparse")
COLMAP_MVS_PATH = Path(f"{SESSION_PATH}/dense")
COLMAP_DB_PATH = Path(f"{SESSION_PATH}/database.db")

!mkdir -p {COLMAP_MVS_PATH}
!mkdir -p {COLMAP_SFM_PATH}

pycolmap.extract_features(COLMAP_DB_PATH, INPUT_PATH)

W20240122 16:37:38.162225  1668 feature_extraction.cc:403] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20240122 16:37:38.162420  1773 misc.cc:198] 
Feature extraction
I20240122 16:37:39.567384  1790 feature_extraction.cc:254] Processed file [1/100]
I20240122 16:37:39.567428  1790 feature_extraction.cc:257]   Name:            000001.jpg
I20240122 16:37:39.567437  1790 feature_extraction.cc:283]   Dimensions:      800 x 800
I20240122 16:37:39.567445  1790 feature_extraction.cc:286]   Camera:          #1 - SIMPLE_RADIAL
I20240122 16:37:39.567451  1790 feature_extraction.cc:289]   Focal Length:    960.00px
I20240122 16:37:39.567462  1790 feature_extraction

In [3]:
# SPARSE RECONSTRUCTION: Step 2/3
pycolmap.match_exhaustive(COLMAP_DB_PATH)

I20240122 16:38:28.282456  1799 misc.cc:198] 
Exhaustive feature matching
I20240122 16:38:28.295940  1799 feature_matching.cc:231] Matching block [1/2, 1/2]
I20240122 16:40:25.595458  1799 feature_matching.cc:46]  in 117.300s
I20240122 16:40:25.712540  1799 feature_matching.cc:231] Matching block [1/2, 2/2]
I20240122 16:42:44.843176  1799 feature_matching.cc:46]  in 139.131s
I20240122 16:42:44.921626  1799 feature_matching.cc:231] Matching block [2/2, 1/2]
I20240122 16:44:59.281576  1799 feature_matching.cc:46]  in 134.360s
I20240122 16:44:59.777310  1799 feature_matching.cc:231] Matching block [2/2, 2/2]
I20240122 16:47:26.078243  1799 feature_matching.cc:46]  in 146.301s
I20240122 16:47:26.497632  1799 timer.cc:91] Elapsed time: 8.970 [minutes]


In [4]:
# SPARSE RECONSTRUCTION: Step 3/3
maps = pycolmap.incremental_mapping(COLMAP_DB_PATH, INPUT_PATH, COLMAP_SFM_PATH)
maps[0].write(COLMAP_SFM_PATH)

I20240122 16:47:35.700744  1898 misc.cc:198] 
Loading database
I20240122 16:47:35.747695  1898 database_cache.cc:54] Loading cameras...
I20240122 16:47:35.751112  1898 database_cache.cc:64]  100 in 0.003s
I20240122 16:47:35.751152  1898 database_cache.cc:72] Loading matches...
I20240122 16:47:35.828817  1898 database_cache.cc:78]  1898 in 0.078s
I20240122 16:47:35.828864  1898 database_cache.cc:94] Loading images...
I20240122 16:47:36.108523  1898 database_cache.cc:143]  100 in 0.280s (connected 100)
I20240122 16:47:36.108589  1898 database_cache.cc:154] Building correspondence graph...
I20240122 16:47:36.137161  1898 database_cache.cc:190]  in 0.029s (ignored 0)
I20240122 16:47:36.137444  1898 timer.cc:91] Elapsed time: 0.006 [minutes]
I20240122 16:47:36.158812  1898 misc.cc:198] 
Finding good initial image pair
I20240122 16:47:36.259881  1898 misc.cc:198] 
Initializing with image pair #17 and #13
I20240122 16:47:36.260766  1898 misc.cc:198] 
Global bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  6.068554e+01    0.00e+00    7.66e+03   0.00e+00   0.00e+00  1.00e+04        0    2.47e-03    6.67e-03
   1  5.865955e+01    2.03e+00    1.55e+03   1.97e+01   1.10e-01  6.79e+03        1    1.98e-03    8.71e-03
   2  4.223130e+01    1.64e+01    4.45e+01   4.03e+00   1.00e+00  2.04e+04        1    4.77e-04    9.22e-03
   3  4.221627e+01    1.50e-02    2.42e+00   2.76e+00   9.91e-01  6.11e+04        1    4.92e-04    9.73e-03
   4  4.221473e+01    1.54e-03    1.12e+00   1.50e+00   9.94e-01  1.83e+05        1    4.41e-04    1.02e-02
   5  4.221466e+01    6.91e-05    7.63e-01   6.63e-02   1.02e+00  5.50e+05        1    4.22e-04    1.06e-02
   6  4.221463e+01    3.59e-05    1.59e+00   7.68e-01   9.43e-01  1.65e+06        1    4.22e-04    1.11e-02
   7  4.221469e+01   -6.24e-05    1.59e+00   2.38e+00  -5.47e-01  8.25e+05        1    1.58e-04    1.13e-02
   8  4.221458e+01    4.71e-

I20240122 16:47:36.312112  1898 misc.cc:205] 
Bundle adjustment report
------------------------
I20240122 16:47:36.312245  1898 bundle_adjustment.cc:942] 
    Residuals : 460
   Parameters : 354
   Iterations : 101
         Time : 0.045706 [s]
 Initial cost : 0.363215 [px]
   Final cost : 0.302914 [px]
  Termination : No convergence

I20240122 16:47:36.312450  1898 incremental_mapper.cc:160] => Filtered observations: 0
I20240122 16:47:36.312461  1898 incremental_mapper.cc:167] => Filtered images: 0
I20240122 16:47:36.327185  1898 misc.cc:198] 
Registering image #14 (3)
I20240122 16:47:36.327236  1898 incremental_mapper.cc:495] => Image sees 88 / 2055 points
I20240122 16:47:36.341008  1898 misc.cc:205] 
Pose refinement report
----------------------
I20240122 16:47:36.341068  1898 bundle_adjustment.cc:942] 
    Residuals : 168
   Parameters : 8
   Iterations : 6
         Time : 0.00294113 [s]
 Initial cost : 0.630711 [px]
   Final cost : 0.385794 [px]
  Termination : Convergence

I202401

   2  2.528686e+02    9.74e-03    2.71e+00   1.53e+00   9.98e-01  9.00e+04        1    3.86e-03    1.22e-02
   3  2.528470e+02    2.17e-02    2.41e+01   3.25e+00   9.57e-01  2.70e+05        1    4.61e-03    1.69e-02
   4  2.528272e+02    1.98e-02    1.20e+02   6.68e+00   4.88e-01  2.70e+05        1    4.16e-03    2.11e-02
   5  2.527912e+02    3.60e-02    6.26e+01   4.79e+00   8.67e-01  4.48e+05        1    4.90e-03    2.60e-02
   6  2.527774e+02    1.38e-02    6.42e+01   4.82e+00   7.08e-01  4.83e+05        1    4.17e-03    3.03e-02
   7  2.527672e+02    1.02e-02    2.59e+01   3.06e+00   9.17e-01  1.15e+06        1    3.96e-03    3.43e-02
   8  2.527645e+02    2.77e-03    2.08e+01   2.74e+00   8.24e-01  1.59e+06        1    3.90e-03    3.82e-02
   9  2.527635e+02    9.39e-04    3.59e+00   1.14e+00   9.82e-01  4.76e+06        1    3.89e-03    4.21e-02
  10  2.527635e+02    5.38e-05    1.20e+00   4.32e-01   9.96e-01  1.43e+07        1    3.96e-03    4.61e-02
  11  2.527635e+02    1.02e-

I20240122 16:47:36.518704  1898 misc.cc:205] 
Bundle adjustment report
------------------------
I20240122 16:47:36.518750  1898 bundle_adjustment.cc:942] 
    Residuals : 4072
   Parameters : 2825
   Iterations : 13
         Time : 0.0546238 [s]
 Initial cost : 0.249205 [px]
   Final cost : 0.249146 [px]
  Termination : Convergence

I20240122 16:47:36.519973  1898 incremental_mapper.cc:175] => Completed observations: 0
I20240122 16:47:36.520251  1898 incremental_mapper.cc:178] => Merged observations: 0
I20240122 16:47:36.520563  1898 incremental_mapper.cc:160] => Filtered observations: 0
I20240122 16:47:36.520586  1898 incremental_mapper.cc:119] => Changed observations: 0.000000
I20240122 16:47:36.520601  1898 incremental_mapper.cc:167] => Filtered images: 0
I20240122 16:47:36.534312  1898 misc.cc:198] 
Registering image #16 (4)
I20240122 16:47:36.534351  1898 incremental_mapper.cc:495] => Image sees 598 / 1998 points
I20240122 16:47:36.571821  1898 misc.cc:205] 
Pose refinement report

...

I20240122 16:48:59.038563  1898 misc.cc:205] 
Bundle adjustment report
------------------------
I20240122 16:48:59.038636  1898 bundle_adjustment.cc:942] 
    Residuals : 172528
   Parameters : 59905
   Iterations : 8
         Time : 1.49772 [s]
 Initial cost : 1.14284 [px]
   Final cost : 0.810869 [px]
  Termination : Convergence

I20240122 16:48:59.105839  1898 incremental_mapper.cc:175] => Completed observations: 22
I20240122 16:48:59.128180  1898 incremental_mapper.cc:178] => Merged observations: 0
I20240122 16:48:59.143069  1898 incremental_mapper.cc:160] => Filtered observations: 552
I20240122 16:48:59.143131  1898 incremental_mapper.cc:119] => Changed observations: 0.006654
I20240122 16:48:59.143155  1898 misc.cc:198] 
Global bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  3.220798e+04    0.00e+00    4.21e+04   0.00e+00   0.00e+00  1.00e+04        0    3.57e-02    3.12e-01
   1  3.001465e+04    2.19e+03    2.40e+04   1.16e+02   9.60e-01  3.00e+04        1    2.70e-01    5.82e-01
   2  2.991511e+04    9.95e+01    2.75e+02   2.90e+01   9.86e-01  9.00e+04        1    1.45e-01    7.27e-01
   3  2.991316e+04    1.94e+00    1.66e+01   5.66e+00   9.98e-01  2.70e+05        1    1.56e-01    8.83e-01
   4  2.991310e+04    5.78e-02    5.33e+00   8.28e-01   9.99e-01  8.10e+05        1    1.56e-01    1.04e+00
   5  2.991310e+04    1.92e-03    1.74e+00   1.37e-01   1.00e+00  2.43e+06        1    1.61e-01    1.20e+00
   6  2.991310e+04    1.15e-05    9.02e-02   1.04e-02   1.01e+00  7.29e+06        1    1.62e-01    1.36e+00


I20240122 16:49:00.610168  1898 misc.cc:205] 
Bundle adjustment report
------------------------
I20240122 16:49:00.610244  1898 bundle_adjustment.cc:942] 
    Residuals : 171468
   Parameters : 59515
   Iterations : 7
         Time : 1.36893 [s]
 Initial cost : 0.433401 [px]
   Final cost : 0.417676 [px]
  Termination : Convergence

I20240122 16:49:00.679090  1898 incremental_mapper.cc:175] => Completed observations: 61
I20240122 16:49:00.701488  1898 incremental_mapper.cc:178] => Merged observations: 0
I20240122 16:49:00.715690  1898 incremental_mapper.cc:160] => Filtered observations: 14
I20240122 16:49:00.715749  1898 incremental_mapper.cc:119] => Changed observations: 0.000875
I20240122 16:49:00.715822  1898 misc.cc:198] 
Global bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.989891e+04    0.00e+00    4.31e+03   0.00e+00   0.00e+00  1.00e+04        0    3.55e-02    3.09e-01
   1  2.982733e+04    7.16e+01    2.28e+01   9.76e+00   1.00e+00  3.00e+04        1    2.61e-01    5.70e-01
   2  2.982726e+04    7.55e-02    1.81e+00   2.65e+00   1.00e+00  9.00e+04        1    1.40e-01    7.11e-01
   3  2.982725e+04    5.23e-03    1.10e+00   5.03e-01   1.00e+00  2.70e+05        1    1.52e-01    8.63e-01
   4  2.982725e+04    7.03e-04    6.21e-01   1.01e-01   9.99e-01  8.10e+05        1    1.57e-01    1.02e+00


I20240122 16:49:01.839308  1898 misc.cc:205] 
Bundle adjustment report
------------------------
I20240122 16:49:01.839385  1898 bundle_adjustment.cc:942] 
    Residuals : 171562
   Parameters : 59506
   Iterations : 5
         Time : 1.02635 [s]
 Initial cost : 0.417462 [px]
   Final cost : 0.416962 [px]
  Termination : Convergence

I20240122 16:49:01.910379  1898 incremental_mapper.cc:175] => Completed observations: 3
I20240122 16:49:01.932457  1898 incremental_mapper.cc:178] => Merged observations: 0
I20240122 16:49:01.946452  1898 incremental_mapper.cc:160] => Filtered observations: 2
I20240122 16:49:01.946518  1898 incremental_mapper.cc:119] => Changed observations: 0.000058
I20240122 16:49:01.946595  1898 incremental_mapper.cc:167] => Filtered images: 0
I20240122 16:49:01.949599  1898 timer.cc:91] Elapsed time: 1.437 [minutes]


In [5]:
# DENSE RECONSTRUCTION: Step 1/3
pycolmap.undistort_images(COLMAP_MVS_PATH, COLMAP_SFM_PATH, INPUT_PATH)

I20240122 16:49:43.224431  1668 images.h:135]  => Reconstruction with 100 images and 19571 points
I20240122 16:49:43.224969  2978 misc.cc:198] 
Image undistortion
I20240122 16:49:43.261833  2978 undistortion.cc:210] Undistorting image [1/100]
I20240122 16:49:43.584400  2978 undistortion.cc:210] Undistorting image [2/100]
I20240122 16:49:43.584446  2978 undistortion.cc:210] Undistorting image [3/100]
I20240122 16:49:43.584465  2978 undistortion.cc:210] Undistorting image [4/100]
I20240122 16:49:43.584477  2978 undistortion.cc:210] Undistorting image [5/100]
I20240122 16:49:43.593442  2978 undistortion.cc:210] Undistorting image [6/100]
I20240122 16:49:43.593477  2978 undistortion.cc:210] Undistorting image [7/100]
I20240122 16:49:43.593487  2978 undistortion.cc:210] Undistorting image [8/100]
I20240122 16:49:43.593494  2978 undistortion.cc:210] Undistorting image [9/100]
I20240122 16:49:43.699529  2978 undistortion.cc:210] Undistorting image [10/100]
I20240122 16:49:43.699661  2978 undi

In [6]:
# DENSE RECONSTRUCTION: Step 3/3
pycolmap.patch_match_stereo(COLMAP_MVS_PATH)  # requires compilation with CUDA

AttributeError: module 'pycolmap' has no attribute 'patch_match_stereo'

In [7]:
# DENSE RECONSTRUCTION: Step 3/3
pycolmap.stereo_fusion(COLMAP_MVS_PATH / "dense.ply", COLMAP_MVS_PATH)

I20240122 16:50:32.829509  2992 misc.cc:205] 
StereoFusion::Options
---------------------
I20240122 16:50:32.829560  2992 fusion.cc:87] mask_path: 
I20240122 16:50:32.829571  2992 fusion.cc:88] max_image_size: -1
I20240122 16:50:32.829578  2992 fusion.cc:89] min_num_pixels: 5
I20240122 16:50:32.829589  2992 fusion.cc:90] max_num_pixels: 10000
I20240122 16:50:32.829599  2992 fusion.cc:91] max_traversal_depth: 100
I20240122 16:50:32.829609  2992 fusion.cc:92] max_reproj_error: 2
I20240122 16:50:32.829625  2992 fusion.cc:93] max_depth_error: 0.01
I20240122 16:50:32.829631  2992 fusion.cc:94] max_normal_error: 10
I20240122 16:50:32.829636  2992 fusion.cc:95] check_num_images: 50
I20240122 16:50:32.829640  2992 fusion.cc:96] use_cache: 0
I20240122 16:50:32.829644  2992 fusion.cc:97] cache_size: 32
I20240122 16:50:32.829649  2992 fusion.cc:100] bbox_min: -3.40282e+38 -3.40282e+38 -3.40282e+38
I20240122 16:50:32.829684  2992 fusion.cc:101] bbox_max: 3.40282e+38 3.40282e+38 3.40282e+38
I202401

Reconstruction(num_reg_images=100, num_cameras=100, num_points3D=0, num_observations=85782)

In [4]:
generate_transforms_script = f"{ROOT}/scripts/third_party/neuralangelo/convert_data_to_json.py"
! python {generate_transforms_script} --data_dir "{SESSION_PATH}" --scene_type "outdoor"

Fraction of images looking at the center: 1.00.
Fraction of images positioned around the center: 1.00.
Valid fraction of concentric images: 1.00.
Writing data to json file:  /mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/data/3_results/LegoTest/transforms.json


In [ ]:
'''
camera = pycolmap.Camera(
    model=camera_model_name_or_id,
    width=width,
    height=height,
    params=params,
)

import pycolmap
reconstruction = pycolmap.Reconstruction("path/to/reconstruction/dir")
print(reconstruction.summary())

for image_id, image in reconstruction.images.items():
    print(image_id, image)

for point3D_id, point3D in reconstruction.points3D.items():
    print(point3D_id, point3D)

for camera_id, camera in reconstruction.cameras.items():
    print(camera_id, camera)

reconstruction.write("path/to/reconstruction/dir/")
'''